In [ ]:
!pip install selenium
import requests
import pickle
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import numpy as np
import math
pd.set_option('display.max_columns', None)

In [ ]:
s = {}
years = np.arange(2010, 2016,1)

def get_month(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')

    month_name = soup.find_all(['div'], {'class': 'filter'})
    month = []
    for i in month_name:
        for m in i:
            if 'href' in str(m):
                month.append(str(m).partition('html">')[-1].partition('</a')[0])
    return month

def get_quarty(url):
    res_box = requests.get(url)
    res_box = res_box.text.replace('<!--','').replace('-->','')
    soup_box = BeautifulSoup(res_box, 'html.parser')
    bs_box = soup_box.find_all(['div','th'],{'class': ['content_grid','center']})
    bs_box = BeautifulSoup(str(bs_box), 'html.parser')
    d = pd.DataFrame()
    d_factor = pd.DataFrame()
    fa = 0
    quarty = 0
    factor = False
    for i in bs_box.find_all(['th','td']):
        
        if 'data-stat' in str(i):
            if 'team' in str(i) and 'href' in str(i):
                if factor:
                    p_factor = {}
                    p_factor['team'] = i.text
                else:
                    p = {}
                    p['team'] = i.text

            if 'td' in str(i):
                if factor:
                    p_factor[i['data-stat']] = i.text
                else:
                    p[i['data-stat']] = i.text
                if 'data-stat="T"' in str(i) or 'data-stat="off_rtg"' in str(i):
                        if factor :
                            if len(p_factor) != 0 and fa <= 1:
                                d_factor = pd.concat([d_factor,pd.DataFrame(p_factor, index = [0])])
                                fa += 1
                        else:
                            if quarty <= 1 and len(p) != 0:          
                                d = pd.concat([d,pd.DataFrame(p, index = [0])])
                                quarty += 1

        if 'data-over-header="Four Factors"' in str(i):
            factor = True
            p_factor = {}
    return d, d_factor

col = ['team', '1', '2', '3', '4', 'T', 'pace', 'efg_pct', 'tov_pct',
       'orb_pct', 'ft_rate', 'off_rtg', 'year', 'month', 'id']

def scrpaer_quarter():
    data = pd.DataFrame()
    for year in years:
        url = 'https://www.basketball-reference.com/leagues/NBA_{}_games.html'.format(year)
        month = get_month(url)
        away_Team = pd.DataFrame()
        home_Team = pd.DataFrame()
        id_Team = pd.DataFrame()
        id_Team_playoff = pd.DataFrame()
        w = 0
        playoff = False
        for m in month:
            url = 'https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html'.format(year,m.lower())
            res = requests.get(url)
            soup = BeautifulSoup(res.text,'html.parser')
            team = soup.find_all(['td','th'])
            for i in team:
                if 'box_score_text' in str(i) and 'td' in str(i):
                    id_ = str(i.a['href']).partition('boxscores/')[-1].partition('.html')[0]
                    html = 'https://www.basketball-reference.com/'+ i.a['href']
                    match_quarty, match_factor = get_quarty(html)
                    match = match_quarty.merge(match_factor, on = 'team')

                    match['year'] = year
                    match['month'] = m
                    match['id'] = id_
                    data = pd.concat([data,match])
                    data = data[col]
        data.reset_index(drop = True).to_csv('nba_team_quarty_{}'.format(year),index = False)

In [ ]:
scrpaer_quarter()